This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [7]:
# get api key from your .env file
import os
from dotenv import load_dotenv  # if missing this module, simply run `pip install python-dotenv`

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import json
import urllib.request

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [12]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = "https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json?api_key=" + API_KEY
data = json.loads(urllib.request.urlopen(url).read().decode('utf8'))

In [75]:
# Used to determine where the keys are located
headers = data['dataset']['column_names']
headers[0]

'Date'

In [20]:
table_headers = data['dataset']['column_names']
for header in table_headers:
    lst(header)

Date
Open
High
Low
Close
Change
Traded Volume
Turnover
Last Price of the Day
Daily Traded Units
Daily Turnover


In [38]:
#Dataset is located within this file.
date_param_1 = '2017-01-01'
date_param_2 = '2017-12-31'

combined = {}

date = []
open_p = []
high = []
low = []
close = []
change = []
traded_volume = []
turnover = []
last_price = []
daily_trade_units = []
daily_turnover = []
    
for lst in data['dataset']['data']:
    if lst[0] >= date_param_1 and lst[0] <= date_param_2:
        date.append(lst[0])
        open_p.append(lst[1])
        high.append(lst[2])
        low.append(lst[3])
        close.append(lst[4])
        change.append(lst[5])
        traded_volume.append(lst[6])
        turnover.append(lst[7])
        last_price.append(lst[8])
        daily_trade_units.append(lst[9])
        daily_turnover.append(lst[10])
        
combined['date'] = date
combined['open'] = open_p
combined['high'] = high
combined['low'] = low
combined['close'] = close
combined['change'] = change
combined['traded_volume'] = traded_volume
combined['turnover'] = turnover
combined['last_price'] = last_price
combined['daily_trade_unnits'] = daily_trade_units
combined['daily_turnover'] = daily_turnover
        

In [92]:
date_index = table_headers.index('Date')
date_index

0

In [93]:
#Refactor of the 2nd approach
date_param_1 = '2017-01-01'
date_param_2 = '2017-12-31'

combined = {}
date_index = table_headers.index('Date')

for number in range(len(table_headers)):
    combined[table_headers[number]] = []
    
for lst in data['dataset']['data']:
    if lst[date_index] >= date_param_1 and lst[date_index] <= date_param_2:
        for number in range(len(table_headers)):
            combined[table_headers[number]].append(lst[number])

In [40]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [41]:
#3 Largest and Smallest Price

print(max(value for value in combined['open'] if isinstance(value, float)))
print(min(value for value in combined['open'] if isinstance(value, float)))

53.11
34.0


In [55]:
#4 Largest change between value of two lists
max_value = 0

for value in range(len(combined['high'])):
    new_value = combined['high'][value] - combined['low'][value]
    if new_value > max_value:
        max_value = new_value
        

print(max_value)


2.8100000000000023


In [58]:
#5 Largest trading value between two days
diff_value = 0

for value in range(len(combined['close'])):
    if value > 0 and combined['close'][value] - combined['close'][value-1] > diff_value:
        diff_value = combined['close'][value] - combined['close'][value-1]
        
print(diff_value)
        

2.559999999999995


In [60]:
#6
round(sum(value for value in combined['traded_volume'] if isinstance(value, float))/len(combined['traded_volume']),2)

89124.34

In [66]:
def median(lst):
    if len(lst) % 2 == 0:
        mid_value = int(len(lst)/2)
        return (lst[mid_value] + lst[mid_value+1])/2
    else:
        mid_value = int((len(lst)+1)/2)
        return lst[mid_value]
    
    
median(combined['traded_volume'])

117165.0